# How to Train Your Own Cone Detection and Key Points Detection Networks

![](https://user-images.githubusercontent.com/22118253/70957091-fe06a480-2042-11ea-8c06-0fcc549fc19a.png)

In this notebook, we will demonstrate 
- how to train your own YOLOv3-based traffic cone detection network and do inference on a video.
- how to train your own KeyPoints detection network and do inference on pictures of traffic cone.

**[Accurate Low Latency Visual Perception for Autonomous Racing: Challenges Mechanisms and Practical Solutions](https://github.com/mit-han-lab/once-for-all)** is an accurate low latency visual perception system introduced by Kieran Strobel, Sibo Zhu, Raphael Chang, and Skanda Koppula.

![](https://user-images.githubusercontent.com/22118253/70950893-e2de6980-202f-11ea-9a16-399579926ee5.gif)

## 1. Preparation
Let's first install all the required packages:

In [20]:
! sudo apt install unzip
! pip3 install imgaug




unzip is already the newest version (6.0-21ubuntu1).
The following packages were automatically installed and are no longer required:
  cuda-10-1 cuda-command-line-tools-10-1 cuda-compiler-10-1 cuda-cudart-10-1
  cuda-cudart-dev-10-1 cuda-cufft-10-1 cuda-cufft-dev-10-1 cuda-cuobjdump-10-1
  cuda-cupti-10-1 cuda-curand-10-1 cuda-curand-dev-10-1 cuda-cusolver-10-1
  cuda-cusolver-dev-10-1 cuda-cusparse-10-1 cuda-cusparse-dev-10-1
  cuda-demo-suite-10-1 cuda-documentation-10-1 cuda-driver-dev-10-1
  cuda-gdb-10-1 cuda-gpu-library-advisor-10-1 cuda-libraries-10-1
  cuda-libraries-dev-10-1 cuda-license-10-1 cuda-memcheck-10-1
  cuda-misc-headers-10-1 cuda-npp-10-1 cuda-npp-dev-10-1 cuda-nsight-10-1
  cuda-nsight-compute-10-1 cuda-nsight-systems-10-1 cuda-nvcc-10-1
  cuda-nvdisasm-10-1 cuda-nvgraph-10-1 cuda-nvgraph-dev-10-1 cuda-nvjpeg-10-1
  cuda-nvjpeg-dev-10-1 cuda-nvml-dev-10-1 cuda-nvprof-10-1 cuda-nvprune-10-1
  cuda-nvrtc-10-1 cuda-nvrtc-dev-10-1 cuda-nvtx-10-1 cuda-nvvp-10-1
  cud

Now, let's build the Cone Detection dataset and the corresponding dataloader. 

In [19]:
print("Downloading Training Dataset")
! wget https://storage.googleapis.com/mit-driverless-open-source/YOLO_Dataset.zip
! unzip YOLO_Dataset.zip
! mv YOLO_Dataset CVC-YOLOv3/dataset/ && cd CVC-YOLOv3/dataset/
! cd CVC-YOLOv3/dataset/ && wget https://storage.googleapis.com/mit-driverless-open-source/yolov3-training/all.csv && cd ../..
! cd CVC-YOLOv3/dataset/ && wget https://storage.googleapis.com/mit-driverless-open-source/yolov3-training/train.csv && cd ../..
! cd CVC-YOLOv3/dataset/ && wget https://storage.googleapis.com/mit-driverless-open-source/yolov3-training/validate.csv && cd ../..
! rm YOLO_Dataset.zip

aset/vid_28_frame_1099.jpg  
  inflating: YOLO_Dataset/vid_209_frame_101.jpg  
  inflating: YOLO_Dataset/vid_31_frame_2312.jpg  
  inflating: YOLO_Dataset/vid_31_frame_1714.jpg  
  inflating: YOLO_Dataset/vid_2_frame_45511.jpg  
  inflating: YOLO_Dataset/vid_40_frame_1050.jpg  
  inflating: YOLO_Dataset/vid_40_frame_842.jpg  
  inflating: YOLO_Dataset/vid_78_frame_192.jpg  
  inflating: YOLO_Dataset/vid_216_frame_159.jpg  
  inflating: YOLO_Dataset/vid_108_frame_351.jpg  
  inflating: YOLO_Dataset/vid_31_frame_3052.jpg  
  inflating: YOLO_Dataset/vid_42_frame_426.jpg  
  inflating: YOLO_Dataset/vid_5_frame_2290.jpg  
  inflating: YOLO_Dataset/vid_2_frame_35752.jpg  
  inflating: YOLO_Dataset/vid_123_frame_8624.jpg  
  inflating: YOLO_Dataset/vid_42_frame_317.jpg
  inflating: YOLO_Dataset/vid_38_frame_629.jpg  
  inflating: YOLO_Dataset/vid_31_frame_2374.jpg  
  inflating: YOLO_Dataset/vid_205_frame_26.jpg  
  inflating: YOLO_Dataset/vid_18_frame_1595.jpg  
  inflating: YOLO_Dataset/vid